In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
dataset_ref = zipfile.ZipFile('/content/drive/MyDrive/archive.zip', 'r')
dataset_ref.extractall('/content')
dataset_ref.close()

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras.utils import image_dataset_from_directory

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the paths for train and validation data
train_dir = '/content/train'
validation_dir = '/content/test'

# Create ImageDataGenerator object
train_datagen = ImageDataGenerator(        
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

# Use flow_from_directory method to create train and validation generators
train_ds = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

validation_ds = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')


Found 15557 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


In [25]:
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [14]:
# # model creation
# model = Sequential()

# model.add(Conv2D(32, kernel_size=(3, 3), padding='valid', activation = 'relu', input_shape = (256, 256, 3)))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size = (2, 2), strides = 2, padding = 'valid'))

# model.add(Conv2D(64, kernel_size=(3, 3), padding='valid', activation = 'relu', input_shape = (256, 256, 3)))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size = (2, 2), strides = 2, padding = 'valid'))

# model.add(Conv2D(128, kernel_size=(3, 3), padding='valid', activation = 'relu', input_shape = (256, 256, 3)))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size = (2, 2), strides = 2, padding = 'valid'))

In [31]:
from keras.layers import Flatten, Dense
from keras.models import Model

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation = 'relu')(x)

# Add a fully connected layer with 23 neurons (one for each class) and softmax activation
predictions = Dense(23, activation = 'softmax')(x)

# Define the functional model
model = Model(inputs = base_model.input, outputs = predictions)

# Freeze the weights of the pre-trained layers in the base model
for layer in base_model.layers:
    layer.trainable = False



In [32]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
history = model.fit(train_ds, validation_data=validation_ds, epochs=30, batch_size=32)

Epoch 1/30
487/487 [==============================] - 311s 638ms/step - loss: 2.7138 - accuracy: 0.1966 - val_loss: 2.6055 - val_accuracy: 0.2231
Epoch 2/30
487/487 [==============================] - 306s 628ms/step - loss: 2.6098 - accuracy: 0.2281 - val_loss: 2.5797 - val_accuracy: 0.2361
Epoch 3/30
487/487 [==============================] - 302s 621ms/step - loss: 2.5638 - accuracy: 0.2349 - val_loss: 2.5090 - val_accuracy: 0.2629
Epoch 4/30
487/487 [==============================] - 300s 616ms/step - loss: 2.5297 - accuracy: 0.2441 - val_loss: 2.5165 - val_accuracy: 0.2679
Epoch 5/30
487/487 [==============================] - 299s 615ms/step - loss: 2.4814 - accuracy: 0.2628 - val_loss: 2.4292 - val_accuracy: 0.2779
Epoch 6/30
487/487 [==============================] - 300s 616ms/step - loss: 2.4637 - accuracy: 0.2669 - val_loss: 2.4383 - val_accuracy: 0.2729
Epoch 7/30
487/487 [==============================] - 300s 616ms/step - loss: 2.4346 - accuracy: 0.2705 - val_loss: 2.4441 -

In [36]:
model.save('/content/drive/MyDrive/my_models/my_model.h5')


In [37]:
model.eval()

AttributeError: ignored